# DATA PROCESSING AND CALCULATION OF xG-ADJUSTED FPL POINTS

## Setup and pre-processing

In [1]:
# give the number of the latest FPL round
latest_gameweek = 13

In [2]:
# import basic libraries
import pandas as pd
import numpy as np
import json
import requests
from scipy.stats import poisson

# allow more data columns to be shown than by default
pd.set_option('max_columns',100)

In [3]:
# import player data 
filepath = '../data/fbref/player_stats_week' + str(latest_gameweek) + '.csv'
playerStats = pd.read_csv(filepath, index_col=0, skiprows=1)
playerStats.head()

,Player,Nation,Pos,Squad,Age,Born,MP,Starts,Min,Gls,Ast,PK,PKatt,CrdY,CrdR,Gls.1,Ast.1,G+A,G-PK,G+A-PK,xG,npxG,xA,xG.1,xA.1,xG+xA,npxG.1,npxG+xA,Matches
Rk,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,Patrick van Aanholt\Patrick-van-Aanholt,nl NED,DF,Crystal Palace,30-111,1990.0,8,7,649,0,1,0,0,1,0,0.00,0.14,0.14,0.00,0.14,0.7,0.7,0.6,0.09,0.08,0.18,0.09,0.18,Matches
2,Tammy Abraham\Tammy-Abraham,eng ENG,FW,Chelsea,23-077,1997.0,12,6,602,3,1,0,0,0,0,0.45,0.15,0.60,0.45,0.60,3.5,3.5,0.9,0.52,0.13,0.65,0.52,0.65,Matches
3,Che Adams\Che-Adams,eng ENG,FW,Southampton,24-158,1996.0,13,13,1118,4,4,0,0,1,0,0.32,0.32,0.64,0.32,0.64,2.7,2.7,3.9,0.22,0.31,0.53,0.22,0.53,Matches
4,Tosin Adarabioyo\Tosin-Adarabioyo,eng ENG,DF,Fulham,23-085,1997.0,9,9,810,0,0,0,0,1,0,0.00,0.00,0.00,0.00,0.00,0.5,0.5,0.2,0.05,0.02,0.07,0.05,0.07,Matches
5,Adrián\Adrian,es ESP,GK,Liverpool,33-350,1987.0,2,2,180,0,0,0,0,0,0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,Matches


In [4]:
# import team data and pre-process
filepath = '../data/fbref/team_stats_week' + str(latest_gameweek) + '.csv'
teamStats = pd.read_csv(filepath, index_col='Squad')
teamStats.head()

,Rk,MP,W,D,L,GF,GA,GDiff,Pts,xG,xGA,xGDiff,xGDiff/90,Last 5,Attendance,Top Team Scorer,Goalkeeper,Notes
Squad,,,,,,,,,,,,,,,,,,
Liverpool,1,13,8,4,1,29,19,10,28,27.1,18.4,8.7,0.67,W D W D W,333.0,Mohamed Salah - 11,Alisson,NaN
Tottenham,2,13,7,4,2,25,12,13,25,18.7,14.2,4.5,0.35,W D W D L,333.0,Son Heung-min - 11,Hugo Lloris,NaN
Southampton,3,13,7,3,3,25,18,7,24,14.4,15.9,-1.5,-0.12,D L W W D,333.0,Danny Ings - 6,Alex McCarthy,NaN
Leicester City,4,13,8,0,5,24,17,7,24,19.9,16.4,3.5,0.27,L L W W L,NaN,Jamie Vardy - 10,Kasper Schmeichel,NaN
Everton,5,13,7,2,4,23,18,5,23,19.7,16.6,3.0,0.23,W L D W W,333.0,Dominic Calvert-Lewin - 11,Jordan Pickford,NaN


In [5]:
# fetch FPL data online
data = json.loads(requests.get('https://fantasy.premierleague.com/api/bootstrap-static/').text)
df = pd.DataFrame(data['elements'])
df.set_index('id',inplace=True)
# this is used later
df_online=1

# fetch data locally
#df = pd.read_csv('../data/fpl/data_week' + str(latest_gameweek) + '.csv', index_col=0)
#df_online=0

df.head()

,assists,bonus,bps,chance_of_playing_next_round,chance_of_playing_this_round,clean_sheets,code,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,creativity,creativity_rank,creativity_rank_type,direct_freekicks_order,direct_freekicks_text,dreamteam_count,element_type,ep_next,ep_this,event_points,first_name,form,goals_conceded,goals_scored,ict_index,ict_index_rank,ict_index_rank_type,in_dreamteam,influence,influence_rank,influence_rank_type,minutes,news,news_added,now_cost,own_goals,penalties_missed,penalties_order,penalties_saved,penalties_text,photo,points_per_game,red_cards,saves,second_name,selected_by_percent,special,squad_number,status,team,team_code,threat,threat_rank,threat_rank_type,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,yellow_cards
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0.0,0.0,0,37605,NaN,,0,0,-2,2,0.0,607,242,NaN,,0,3,0.0,0.0,0,Mesut,0.0,0,0,0.0,607,242,False,0.0,607,242,0,Not included in Arsenal's 25-man Premier Leagu...,2020-10-20T22:30:18.118477Z,68,0,0,NaN,0,,37605.jpg,0.0,0,0,Özil,0.6,False,None,n,1,3,0.0,607,242,0,3380,4,52315,141,0.0,0.0,Özil,0
2,0,0,0,0.0,0.0,0,39476,NaN,,0,0,-2,2,0.0,550,210,NaN,,0,2,0.0,0.0,0,Sokratis,0.0,0,0,0.0,563,210,False,0.0,561,210,0,Not included in Arsenal's 25-man Premier Leagu...,2020-10-21T10:30:18.546407Z,48,0,0,NaN,0,,39476.jpg,0.0,0,0,Papastathopoulos,0.1,False,None,n,1,3,0.0,534,203,0,10240,1,18723,36,0.0,0.0,Sokratis,0
3,0,0,56,100.0,75.0,0,41270,NaN,,-1,1,-1,1,23.1,283,86,4.0,,0,2,0.7,0.5,1,David,0.2,7,0,12.2,315,109,False,77.2,256,100,389,,2020-11-26T23:30:22.462494Z,54,0,0,NaN,0,,41270.jpg,1.1,0,0,Luiz Moreira Marinho,0.8,False,None,a,1,3,22.0,283,88,8,37524,539,84985,712,0.0,1.5,David Luiz,0
4,1,4,146,100.0,100.0,3,54694,NaN,,-1,1,-6,6,201.2,46,34,NaN,,1,3,3.9,3.9,10,Pierre-Emerick,3.4,16,3,76.0,30,18,False,186.6,108,39,1166,,2020-10-08T09:00:12.305506Z,114,1,0,1.0,0,,54694.jpg,3.6,0,0,Aubameyang,8.4,False,None,a,1,3,372.0,20,11,47,484330,3325,3051148,13721,0.3,4.1,Aubameyang,2
5,0,0,3,100.0,100.0,0,58822,NaN,,0,0,-4,4,0.0,535,206,NaN,,0,2,0.7,0.7,1,Cédric,0.2,0,0,0.0,550,206,False,0.0,548,206,5,,2020-09-23T09:00:14.881983Z,46,0,0,NaN,0,,58822.jpg,1.0,0,0,Soares,0.2,False,None,a,1,3,0.0,513,197,1,3618,73,33641,178,0.0,0.2,Cédric,0


In [6]:
# assign proper team names for each player
team_names = np.sort(teamStats.index)
# for some reason the fpl team numbers are not alphabetical with leeds and leicester...
if (team_names[8]=='Leeds United') & (team_names[9]=='Leicester City'):
    team_names[8] = 'Leicester City'
    team_names[9] = 'Leeds United'
df['team_name'] = team_names[df['team']-1]

## Probability to keep a clean sheet

Here, we estimate for each team the probability that the team keeps a clean sheet (against average opposition). We do this by first calculating the expected goals allowed per game for each team. Then, we assume that conceding goals follows a Poisson distribution, from which we then get the desired probability.

In [7]:
teamStats['xG per game'] = teamStats['xG'] / teamStats['MP']
teamStats['xGA per game'] = teamStats['xGA'] / teamStats['MP']
teamStats['probability no goals allowed'] = poisson.pmf(0,teamStats['xGA per game'])
teamStats

,Rk,MP,W,D,L,GF,GA,GDiff,Pts,xG,xGA,xGDiff,xGDiff/90,Last 5,Attendance,Top Team Scorer,Goalkeeper,Notes,xG per game,xGA per game,probability no goals allowed
Squad,,,,,,,,,,,,,,,,,,,,,
Liverpool,1,13,8,4,1,29,19,10,28,27.1,18.4,8.7,0.67,W D W D W,333.0,Mohamed Salah - 11,Alisson,NaN,2.084615,1.415385,0.242832
Tottenham,2,13,7,4,2,25,12,13,25,18.7,14.2,4.5,0.35,W D W D L,333.0,Son Heung-min - 11,Hugo Lloris,NaN,1.438462,1.092308,0.335442
Southampton,3,13,7,3,3,25,18,7,24,14.4,15.9,-1.5,-0.12,D L W W D,333.0,Danny Ings - 6,Alex McCarthy,NaN,1.107692,1.223077,0.294323
Leicester City,4,13,8,0,5,24,17,7,24,19.9,16.4,3.5,0.27,L L W W L,NaN,Jamie Vardy - 10,Kasper Schmeichel,NaN,1.530769,1.261538,0.283218
Everton,5,13,7,2,4,23,18,5,23,19.7,16.6,3.0,0.23,W L D W W,333.0,Dominic Calvert-Lewin - 11,Jordan Pickford,NaN,1.515385,1.276923,0.278894
Manchester Utd,6,12,7,2,3,22,19,3,23,16.2,14.6,1.6,0.15,W W W D W,NaN,Bruno Fernandes - 7,David de Gea,NaN,1.350000,1.216667,0.296216
Chelsea,7,13,6,4,3,26,14,12,22,21.8,12.9,9.0,0.69,W D W L L,333.0,Timo Werner Kurt Zouma - 4,Edouard Mendy,NaN,1.676923,0.992308,0.370720
West Ham,8,13,6,3,4,21,16,5,21,17.7,17.7,0.0,0.00,W W L W D,NaN,Jarrod Bowen - 4,Łukasz Fabiański,NaN,1.361538,1.361538,0.256266
Manchester City,9,12,5,5,2,18,12,6,20,20.2,11.2,9.0,0.75,L W W D D,NaN,Riyad Mahrez - 4,Ederson,NaN,1.683333,0.933333,0.393241


## xG-adjusted points

Next, we determine for each player their 'adjusted points'. To do this, we first subtract for each player all the points they have accumulated through goals, assists and clean sheets. Then, we add points for each player based on their expected goals, assists and clean sheets. This gives a much improved estimate of each player's true point generating capability. 

A player who gives an assist that directly leads to a shot, is assigned the xG-value of the shot in xA (expected assists), i.e. xA is a measure of 'goal assists'. In FPL, however, the definition of an assist is somewhat more relaxed, e.g. goals resulting from a rebound of parried shot will award an assist to the player making the initial shot. For this reason, we calculate the total number of assists awarded in FPL and the total sum of xA of all players and get an estimate of the proportion of assists that xA covers in FPL. Then, we modify that proportion of players' assists based on their xA.

In [8]:
# auxiliary information and variables
df['points_per_game'] = df['points_per_game'].astype(float)
df['games played'] = df['total_points']/df['points_per_game']
xA_proportion = playerStats['xA'].sum()/df['assists'].sum()
xA_proportion

0.6645259938837921

In [9]:
df_previous_week = pd.read_csv('../data/fpl/data_week' + str(latest_gameweek-1) + '.csv', index_col=0)
if latest_gameweek == 1:
    df_previous_week['minutes'] = 0
    df_previous_week['xG'] = np.nan
    df_previous_week['xA'] = np.nan
df_previous_week.head()

,assists,bonus,bps,chance_of_playing_next_round,chance_of_playing_this_round,clean_sheets,code,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,creativity,creativity_rank,creativity_rank_type,direct_freekicks_order,direct_freekicks_text,dreamteam_count,element_type,ep_next,ep_this,event_points,first_name,form,goals_conceded,goals_scored,ict_index,ict_index_rank,ict_index_rank_type,in_dreamteam,influence,influence_rank,influence_rank_type,minutes,news,news_added,now_cost,own_goals,penalties_missed,penalties_order,penalties_saved,penalties_text,photo,points_per_game,red_cards,saves,second_name,selected_by_percent,special,...,xG_week6,goals_week6,xGA_week6,cleansheet_week6,xA_week6,assists_week6,xPoints week 7,xG_week7,goals_week7,xGA_week7,cleansheet_week7,xA_week7,assists_week7,xPoints week 8,xG_week8,goals_week8,xGA_week8,cleansheet_week8,xA_week8,assists_week8,xPoints week 9,xG_week9,goals_week9,xGA_week9,cleansheet_week9,xA_week9,assists_week9,xPoints week 10,xG_week10,goals_week10,xGA_week10,cleansheet_week10,xA_week10,assists_week10,xPoints week 11,xG_week11,goals_week11,xGA_week11,cleansheet_week11,xA_week11,assists_week11,form 10,form 5,next1_xP,next5_xP,next10_xP,value,valuePoints metric,xG+xA,xG+xA per game
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0.0,0.0,0,37605,NaN,NaN,0,0,-2,2,0.0,607,242,NaN,NaN,0,3,0.0,0.0,0,Mesut,0.0,0,0,0.0,607,242,False,0.0,607,242,0,Not included in Arsenal's 25-man Premier Leagu...,2020-10-20T22:30:18.118477Z,68,0,0,NaN,0,NaN,37605.jpg,0.0,0,0,Özil,0.6,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,0,0,0.0,0.0,0,39476,NaN,NaN,0,0,-1,1,0.0,562,212,NaN,NaN,0,2,0.0,0.0,0,Sokratis,0.0,0,0,0.0,573,212,False,0.0,571,212,0,Not included in Arsenal's 25-man Premier Leagu...,2020-10-21T10:30:18.546407Z,49,0,0,NaN,0,NaN,39476.jpg,0.0,0,0,Papastathopoulos,0.1,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,0,52,75.0,75.0,0,41270,NaN,NaN,0,0,0,0,23.1,276,84,4.0,NaN,0,2,0.4,0.8,0,David,0.0,7,0,11.7,307,104,False,72.0,249,94,364,Head injury - 75% chance of playing,2020-11-26T23:30:22.462494Z,55,0,0,NaN,0,NaN,41270.jpg,1.2,0,0,Luiz Moreira Marinho,0.8,False,...,0.0,0.0,1.2,0.0,0.0,0.0,NaN,NaN,NaN,0.3,0.0,NaN,NaN,NaN,NaN,NaN,1.8,0.0,NaN,NaN,NaN,NaN,NaN,2.5,0.0,NaN,NaN,0.000000,0.0,0.0,1.4,0.0,0.0,0.0,NaN,NaN,NaN,0.5,0.0,NaN,NaN,2.212092,0.000000,2.790035,2.452258,2.548930,0.440106,1.032139,0.1,0.017143
4,1,1,122,100.0,100.0,3,54694,NaN,NaN,0,0,-5,5,189.3,44,33,NaN,NaN,0,3,2.3,2.8,0,Pierre-Emerick,1.8,15,2,68.3,31,18,False,148.8,143,55,1076,NaN,2020-10-08T09:00:12.305506Z,115,1,0,1.0,0,NaN,54694.jpg,3.1,0,0,Aubameyang,8.8,False,...,0.3,0.0,1.2,0.0,0.1,0.0,7.540818,0.9,1.0,0.3,1.0,0.1,0.0,2.765299,0.0,0.0,1.8,0.0,0.2,0.0,3.282085,0.3,0.0,2.5,1.0,-0.1,0.0,4.246597,0.4,0.0,1.4,0.0,0.0,0.0,3.606531,0.2,0.0,0.5,0.0,0.0,0.0,3.586421,3.121416,3.726013,3.555358,3.595918,0.313523,1.063210,4.2,0.351892
5,0,0,0,100.0,100.0,0,58822,NaN,NaN,0,0,-4,4,0.0,535,206,NaN,NaN,0,2,0.5,1.0,0,Cédric,0.0,0,0,0.0,549,206,False,0.0,547,206,0,NaN,2020-09-23T09:00:14.881983Z,46,0,0,NaN,0,NaN,58822.jpg,0.0,0,0,Soares,0.2,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
if latest_gameweek > 1:
    filepath = '../data/fbref/team_stats_week' + str(latest_gameweek-1) + '.csv'
    teamStats_previous_week = pd.read_csv(filepath, index_col='Squad')
    display(teamStats_previous_week.head())
elif latest_gameweek == 1:
    teamStats_previous_week = pd.DataFrame(np.nan, index=team_names, columns=['xGA'])
    display(teamStats_previous_week)

,Rk,MP,W,D,L,GF,GA,GDiff,Pts,xG,xGA,xGDiff,xGDiff/90,Last 5,Attendance,Top Team Scorer,Goalkeeper,Notes,xG per game,xGA per game,probability no goals allowed
Squad,,,,,,,,,,,,,,,,,,,,,
Tottenham,1,12,7,4,1,24,10,14,25,17.4,13.2,4.2,0.35,W W D W D,333.0,Son Heung-min - 10,Hugo Lloris,NaN,1.450000,1.100000,0.332871
Liverpool,2,12,7,4,1,27,18,9,25,25.9,17.1,8.8,0.73,D W D W D,333.0,Mohamed Salah - 10,Alisson,NaN,2.158333,1.425000,0.240508
Leicester City,3,12,8,0,4,24,15,9,24,19.0,14.9,4.1,0.34,W L L W W,NaN,Jamie Vardy - 10,Kasper Schmeichel,NaN,1.583333,1.241667,0.288902
Southampton,4,12,7,2,3,24,17,7,23,13.3,15.1,-1.8,-0.15,W D L W W,333.0,Danny Ings - 6,Alex McCarthy,NaN,1.108333,1.258333,0.284127
Chelsea,5,12,6,4,2,25,12,13,22,20.6,12.4,8.2,0.68,W W D W L,333.0,Timo Werner Kurt Zouma - 4,Edouard Mendy,NaN,1.716667,1.033333,0.355819


In [11]:
def points_for_goal(position):
    if position < 3:
        return 6
    elif position==3:
        return 5
    elif position==4:
        return 4

def points_for_clean_sheet(position):
    if position < 3:
        return 4
    elif position==3:
        return 1
    elif position==4:
        return 0

In [12]:
# column names to be used below
xPoints_column = 'xPoints week ' + str(latest_gameweek)
xG_column = 'xG_week' + str(latest_gameweek) 
goals_column = 'goals_week' + str(latest_gameweek) 
xGA_column = 'xGA_week' + str(latest_gameweek) 
cleansheet_column = 'cleansheet_week' + str(latest_gameweek) 
xA_column = 'xA_week' + str(latest_gameweek) 
assists_column = 'assists_week' + str(latest_gameweek) 

In [13]:
def incorporate_xG(indicator, ix):
    xG = playerStats.loc[indicator, 'xG'].values[0]  
    penalty_attempts = playerStats.loc[indicator,'PKatt'].values[0]
    position = df.loc[ix, 'element_type']
    
    # 0.24 is the probability to miss a penalty, which incurs -2 points
    df.loc[ix, 'adjusted points'] =  df.loc[ix, 'total_points'] \
                                        - points_for_goal(position) * (df.loc[ix, 'goals_scored'] - xG) \
                                        - 2 * 0.24 * penalty_attempts    
    df.loc[ix, 'xG'] = xG
    df.loc[ix, 'xG_points'] = points_for_goal(position)*xG - 2 * 0.24 * penalty_attempts
    
    try:
        if df.loc[ix,'minutes'] > df_previous_week.loc[ix,'minutes']:
            if ~np.isnan(df_previous_week.loc[ix, 'xG']):
                df.loc[ix, xG_column] = df.loc[ix, 'xG'] - df_previous_week.loc[ix, 'xG']
                df.loc[ix, goals_column] = df.loc[ix, 'goals_scored'] - df_previous_week.loc[ix, 'goals_scored']
                df.loc[ix, xPoints_column] = df.loc[ix, 'event_points'] - points_for_goal(position) * \
                                                        (df.loc[ix, goals_column] - df.loc[ix, xG_column])
            else:
                df.loc[ix, xG_column] = xG
                df.loc[ix, goals_column] = df.loc[ix, 'goals_scored']
                df.loc[ix, xPoints_column] = df.loc[ix, 'event_points'] - points_for_goal(position) * \
                                                        (df.loc[ix, goals_column] - df.loc[ix, xG_column])
    except KeyError:  
        print(str(ix) + ' is a new index. (xG)')

In [14]:
# always run 'team_xGA' AFTER 'incorporate_xG'
def team_xGA(indicator, ix):
    team = team_names[df.loc[ix, 'team']-1]
    clean_sheets = df.loc[ix, 'clean_sheets']
    probability_cleanSheet = teamStats.loc[team, 'probability no goals allowed']
    position = df.loc[ix, 'element_type']
    
    df.loc[ix, 'adjusted points'] =  df.loc[ix, 'adjusted points'] - points_for_clean_sheet(position) * \
                                ( df.loc[ix, 'clean_sheets'] - probability_cleanSheet*df.loc[ix, 'games played'] )
    df.loc[ix, 'clean_sheet_points'] = points_for_clean_sheet(position) * \
                                       df.loc[ix, 'games played'] * probability_cleanSheet
    try:
        if ~np.isnan(teamStats_previous_week.loc[team, 'xGA']):            
            df.loc[ix, xGA_column] = teamStats.loc[team, 'xGA'] - teamStats_previous_week.loc[team, 'xGA']
            df.loc[ix, cleansheet_column] = df.loc[ix, 'clean_sheets'] - df_previous_week.loc[ix, 'clean_sheets']
        else:
            df.loc[ix, xGA_column] = teamStats.loc[team, 'xGA']
            df.loc[ix, cleansheet_column] = df.loc[ix, 'clean_sheets']
        
        if df.loc[ix,'minutes'] - df_previous_week.loc[ix,'minutes'] >= 60:
            probability_cleansheet_thisweek = poisson.pmf(0,df.loc[ix, xGA_column])
            df.loc[ix, xPoints_column] = df.loc[ix, xPoints_column] - points_for_clean_sheet(position) * \
                                            (df.loc[ix, cleansheet_column] - probability_cleansheet_thisweek)
        
    except KeyError:  
        print(str(ix) + ' is a new index. (xGA)')

In [15]:
# always run 'xA' AFTER 'incorporate_xG'
def xA(indicator, ix):
    xA = playerStats.loc[indicator, 'xA'].values[0]
    df.loc[ix, 'adjusted points'] =  df.loc[ix, 'adjusted points'] -  3 * (xA_proportion*df.loc[ix, 'assists'] - xA)
    df.loc[ix, 'xA'] = xA
    df.loc[ix, 'xA_points'] = 3*xA
    try:
        if df.loc[ix,'minutes'] > df_previous_week.loc[ix,'minutes']:
            if ~np.isnan(df_previous_week.loc[ix, 'xA']):
                df.loc[ix, xA_column] = df.loc[ix, 'xA'] - df_previous_week.loc[ix, 'xA']
                df.loc[ix, assists_column] = df.loc[ix, 'assists'] - df_previous_week.loc[ix, 'assists']
                df.loc[ix, xPoints_column] = df.loc[ix, xPoints_column] - \
                                    3 * (xA_proportion*df.loc[ix, assists_column] - df.loc[ix, xA_column])
            else:
                df.loc[ix, xA_column] = xA
                df.loc[ix, assists_column] = df.loc[ix, 'assists']
                df.loc[ix, xPoints_column] = df.loc[ix, xPoints_column] - \
                                            3*(xA_proportion*df.loc[ix, assists_column] - df.loc[ix, xA_column])
    except KeyError:  
        print(str(ix) + ' is a new index. (xA)')

In [16]:
def calculateAdjustedPoints(indicator, index):
    incorporate_xG(indicator, index)
    team_xGA(indicator, index)
    xA(indicator, index)

## Main loop for assigning adjusted points

Below is the main loop where we calculate adjusted points for each player. Calculation of the adjusted points itself is straightforward, but there is some work required to match players in two different data sets. Comparing player names in both data sets gives unique matches in many cases, but some special cases need to be covered through individual solutions.

In [17]:
for ix in df[df['minutes']>0].index: 
    # player name in FPL data
    name = df.loc[ix, 'web_name'].lower().replace(' ', '').replace('-', '').replace('ü', 'u').replace('ö', 'o').\
                        replace('ä', 'a')
    # find FBREF data indexes where player name contains 'name'
    indicator = playerStats['Player'].str.lower().str.replace(' ', '').str.replace('-', '').str.contains(name)
    # if unique match is found, we can calculate 'adjusted points'
    if playerStats.loc[indicator].shape[0]==1:
        calculateAdjustedPoints(indicator, ix)
    # if no match is found
    elif playerStats.loc[indicator].shape[0]==0:
        # try matching based on first name, helps in some cases
        first_name = df.loc[ix, 'first_name'].lower().replace(' ', '').replace('-', '').replace('ü', 'u')\
                                                                    .replace('ö', 'o').replace('ä', 'a')
        first_name_indicator = playerStats['Player'].str.lower().str.replace(' ', '').str.replace('-', '')\
                                                                            .str.contains(first_name)
        # if unique match is found, we can calculate 'adjusted points'
        if playerStats.loc[first_name_indicator].shape[0]==1:
            calculateAdjustedPoints(first_name_indicator, ix)
        # here we deal with some special cases individually
        else:
            names = ['rodrigo','elliott','mattylongstaff',"n'lundulu",'carlosvinicius']
            if name == 'rodrigo':
                exceptional_case_indicator = \
                (playerStats['Player'].str.lower().str.replace(' ', '').str.replace('-', '').str.contains('rodri')) & \
                (~playerStats['Player'].str.lower().str.replace(' ', '').str.replace('-', '').str.contains('rodriguez'))
                calculateAdjustedPoints(exceptional_case_indicator, ix)
            elif name == 'elliott':
                exceptional_case_indicator = (playerStats['Player'].str.lower().str.replace(' ', '')\
                                              .str.replace('-', '').str.contains('elliot'))
                calculateAdjustedPoints(exceptional_case_indicator, ix)
            elif name == 'mattylongstaff':
                exceptional_case_indicator = (playerStats['Player'].str.lower().str.contains('matthew longstaff'))
                calculateAdjustedPoints(exceptional_case_indicator, ix)
            elif name == "n'lundulu":
                exceptional_case_indicator = playerStats['Player'].str.lower().str.replace(' ', '')\
                                                .str.replace('-', '').str.contains('lundulu')
                calculateAdjustedPoints(exceptional_case_indicator, ix)
            elif name == 'carlosvinicius':
                exceptional_case_indicator = playerStats['Player'].str.lower().str.replace(' ', '')\
                                                .str.replace('-', '').str.contains('carlos')
                calculateAdjustedPoints(exceptional_case_indicator, ix)
            else:
                print(str(ix) + ': no player found.')
    # if more than one matches were found in the original comparison
    elif playerStats.loc[indicator].shape[0]>1:
        # try matching full name
        full_name = df.loc[ix, 'first_name'].lower().replace(' ', '').replace('-', '') \
            + df.loc[ix, 'second_name'].lower().replace(' ', '').replace('-', '')
        full_name_indicator = playerStats['Player'].str.lower().str.replace(' ', '').str.replace('-', '')\
                                                                            .str.contains(full_name)
        # if unique match is found, we can calculate 'adjusted points'
        if playerStats.loc[full_name_indicator].shape[0]==1:
            calculateAdjustedPoints(full_name_indicator, ix)
        # here we try to match players based both on their name and team
        else:
            team = team_names[df.loc[ix, 'team']-1]
            team_indicator = playerStats['Squad']==team
            if playerStats.loc[indicator & team_indicator].shape[0]==1:
                calculateAdjustedPoints(indicator & team_indicator, ix)
            # special case
            else:
                if name=='son':
                    exceptional_case_indicator = playerStats['Player'].str.lower().str.replace(' ', '')\
                                                .str.replace('-', '').str.contains('heung')
                    calculateAdjustedPoints(exceptional_case_indicator, ix)
                else:
                    print(str(ix) + ': non-unique name.')
    else:
        print(str(ix) + 'Player not found')

In [18]:
df['adjusted points per game'] = df['adjusted points'] / df['games played']

if (latest_gameweek > 1) & (df_online==1):
    column_list = []
    for i in range(1,latest_gameweek):
        column_list.append('xPoints week ' + str(i))
        column_list.append('xG_week' + str(i))
        column_list.append('goals_week' + str(i))
        column_list.append('xGA_week' + str(i))
        column_list.append('cleansheet_week' + str(i))
        column_list.append('xA_week' + str(i))
        column_list.append('assists_week' + str(i))
    df = df.join( df_previous_week[column_list])

df.head()

,assists,bonus,bps,chance_of_playing_next_round,chance_of_playing_this_round,clean_sheets,code,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,creativity,creativity_rank,creativity_rank_type,direct_freekicks_order,direct_freekicks_text,dreamteam_count,element_type,ep_next,ep_this,event_points,first_name,form,goals_conceded,goals_scored,ict_index,ict_index_rank,ict_index_rank_type,in_dreamteam,influence,influence_rank,influence_rank_type,minutes,news,news_added,now_cost,own_goals,penalties_missed,penalties_order,penalties_saved,penalties_text,photo,points_per_game,red_cards,saves,second_name,selected_by_percent,special,...,assists_week5,xPoints week 6,xG_week6,goals_week6,xGA_week6,cleansheet_week6,xA_week6,assists_week6,xPoints week 7,xG_week7,goals_week7,xGA_week7,cleansheet_week7,xA_week7,assists_week7,xPoints week 8,xG_week8,goals_week8,xGA_week8,cleansheet_week8,xA_week8,assists_week8,xPoints week 9,xG_week9,goals_week9,xGA_week9,cleansheet_week9,xA_week9,assists_week9,xPoints week 10,xG_week10,goals_week10,xGA_week10,cleansheet_week10,xA_week10,assists_week10,xPoints week 11,xG_week11,goals_week11,xGA_week11,cleansheet_week11,xA_week11,assists_week11,xPoints week 12,xG_week12,goals_week12,xGA_week12,cleansheet_week12,xA_week12,assists_week12
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0.0,0.0,0,37605,NaN,,0,0,-2,2,0.0,607,242,NaN,,0,3,0.0,0.0,0,Mesut,0.0,0,0,0.0,607,242,False,0.0,607,242,0,Not included in Arsenal's 25-man Premier Leagu...,2020-10-20T22:30:18.118477Z,68,0,0,NaN,0,,37605.jpg,0.0,0,0,Özil,0.6,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,0,0,0.0,0.0,0,39476,NaN,,0,0,-2,2,0.0,550,210,NaN,,0,2,0.0,0.0,0,Sokratis,0.0,0,0,0.0,563,210,False,0.0,561,210,0,Not included in Arsenal's 25-man Premier Leagu...,2020-10-21T10:30:18.546407Z,48,0,0,NaN,0,,39476.jpg,0.0,0,0,Papastathopoulos,0.1,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,0,56,100.0,75.0,0,41270,NaN,,-1,1,-1,1,23.1,283,86,4.0,,0,2,0.7,0.5,1,David,0.2,7,0,12.2,315,109,False,77.2,256,100,389,,2020-11-26T23:30:22.462494Z,54,0,0,NaN,0,,41270.jpg,1.1,0,0,Luiz Moreira Marinho,0.8,False,...,0.0,1.000000,0.0,0.0,1.2,0.0,0.0,0.0,NaN,NaN,NaN,0.3,0.0,NaN,NaN,NaN,NaN,NaN,1.8,0.0,NaN,NaN,NaN,NaN,NaN,2.5,0.0,NaN,NaN,0.000000,0.0,0.0,1.4,0.0,0.0,0.0,NaN,NaN,NaN,0.5,0.0,NaN,NaN,NaN,NaN,NaN,0.9,0.0,NaN,NaN
4,1,4,146,100.0,100.0,3,54694,NaN,,-1,1,-6,6,201.2,46,34,NaN,,1,3,3.9,3.9,10,Pierre-Emerick,3.4,16,3,76.0,30,18,False,186.6,108,39,1166,,2020-10-08T09:00:12.305506Z,114,1,0,1.0,0,,54694.jpg,3.6,0,0,Aubameyang,8.4,False,...,0.0,4.101194,0.3,0.0,1.2,0.0,0.1,0.0,7.540818,0.9,1.0,0.3,1.0,0.1,0.0,2.765299,0.0,0.0,1.8,0.0,0.2,0.0,3.282085,0.3,0.0,2.5,1.0,-0.1,0.0,4.246597,0.4,0.0,1.4,0.0,0.0,0.0,3.606531,0.2,0.0,0.5,0.0,0.0,0.0,1.70657,0.2,0.0,0.9,0.0,0.1,0.0
5,0,0,3,100.0,100.0,0,58822,NaN,,0,0,-4,4,0.0,535,206,NaN,,0,2,0.7,0.7,1,Cédric,0.2,0,0,0.0,550,206,False,0.0,548,206,5,,2020-09-23T09:00:14.881983Z,46,0,0,NaN,0,,58822.jpg,1.0,0,0,Soares,0.2,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
# give a sorted list showing the players with highest 'adjusted points per game'
df[['web_name', 'games played','total_points', 'points_per_game','adjusted points','adjusted points per game']]\
                                .sort_values(by='adjusted points per game', ascending=False)

,web_name,games played,total_points,points_per_game,adjusted points,adjusted points per game
id,,,,,,
272,De Bruyne,10.909091,60,5.5,77.788431,7.130606
254,Salah,12.000000,102,8.5,85.333252,7.111104
388,Kane,12.976190,109,8.4,92.104220,7.097940
224,Vardy,12.027027,89,7.4,80.352110,6.680962
302,Fernandes,11.971831,85,7.1,76.658357,6.403227
390,Son,12.954545,114,8.8,80.571180,6.219530
202,Bamford,13.064516,81,6.2,77.419266,5.925919
141,Zaha,11.014493,76,6.9,64.551594,5.860605
506,Wilson,11.060606,73,6.6,64.085688,5.794049


In [20]:
# save data
filepath = '../data/fpl/data_week' + str(latest_gameweek) + str('.csv')
df.to_csv(filepath)

filepath = '../data/fbref/team_stats_week' + str(latest_gameweek) + '.csv'
teamStats.to_csv(filepath)

Below we check how well the total xG matches the total scored goals.

In [21]:
total_xG = playerStats['xG'].sum()
total_goals = df['goals_scored'].sum()
print('Total goals: ' + str(total_goals))
print('Total xG: ' + str(total_xG))
print('goals per xG: ' + str(total_goals/total_xG))

Total goals: 351
Total xG: 338.6
goals per xG: 1.0366213821618429
